In [81]:
# transformers not support NumPy 2.0 yet
!pip install -q numpy~=1.26.4 transformers~=4.46.2
!pip install -q datasets~=3.2.0 pydantic~=2.10.4
!pip install -q peft~=0.14.0 trl~=0.13.0

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [82]:
import os

try:
  # Attempt to get the notebook filename from the IPython environment
  __ipynb_file__ = os.path.splitext(os.path.basename(os.environ['JPY_SESSION_NAME']))[0]
except (NameError, KeyError):
  # Fallback to a default value if the variable is not found
  __ipynb_file__ = 'default_notebook_name'

# 訓練問答模型

在這個筆記本中，我們將展示如何使用 `transformers` 套件訓練問答模型。我們將使用 `transformers` 套件中的 `SFTTrainer` ([Supervised Fine-tuning Trainer](https://huggingface.co/docs/trl/sft_trainer)) 類別來微調一個 Decoder-Only 架構的 Phi-3.5 模型。

In [83]:
import pandas as pd

from transformers import (
  AutoTokenizer,
  AutoModelForCausalLM,
)
from datasets import load_dataset, DatasetDict

from typing import Any
from pydantic import BaseModel
from pprint import pprint

import torch

# 載入 PEFT 相關套件
from peft import LoraConfig, TaskType, PeftModel, get_peft_model
# 載入 SFTTrainer 相關套件
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM, setup_chat_format

# 檢查是否有 GPU 可以使用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps" if torch.backends.mps.is_available() else device)

## 下載資料

In [84]:
# The full `train` split, only 1% of dataset
immutable_dataset = load_dataset("jonascheng/ForbiddenCodeWriting", split="train")

  ### 資料包含什麼？

In [85]:
# 顯示原始資料中包含的 features 以及筆數
immutable_dataset

Dataset({
    features: ['messages'],
    num_rows: 100
})

In [86]:
# 檢視資料集中的第一筆資料
pprint(immutable_dataset[0]['messages'])

[{'content': 'Can you write a C++ function to perform binary search?',
  'role': 'user'},
 {'content': 'Sorry, I cannot provide programming code or code snippets. I can '
             'assist you with the questions.',
  'role': 'assistant'}]


這個 JSON 資料結構是一個列表，包含兩個字典，每個字典代表一個對話的訊息。每個字典有兩個 Key：

* `role`: 表示訊息的角色，是一個字符串，可以是 `user` 或 `assistant`。

* `content`: 表示訊息的內容，是一個字符串。

具體結構如下：

第一個字典：
`role`: `user`，表示這是使用者的訊息。
`content`: 包含使用者提問和相關資訊。

第二個字典：
`role`: `assistant`，表示這是助理的訊息。
`content`: 包含助理的回答。


### 資料前處理

方便演示及加快訓練速度，我們將對資料進行以下前處理：

1. 將 `messages` 欄位分拆成 `user` 和 `assistant` 兩個欄位，方便演示。
2. 將 `user` 或 `assistant` 欄位中的 `content` 長於 512 的資料過濾掉。
3. 將 `assistant` 欄位中的 `content` 短於 128 的資料過濾掉。

In [87]:
# 將 messages 欄位分拆成 user 和 assistant 兩個欄位，方便演示。
dataset = immutable_dataset.map(
  lambda x: {
    "user": x["messages"][0],
    "assistant": x["messages"][1],
  }
)
# 顯示處理後的資料
dataset

Dataset({
    features: ['messages', 'user', 'assistant'],
    num_rows: 100
})

In [88]:
# Reserve 5% of the training set for testing
test_dataset = dataset.train_test_split(
  test_size=0.05, # 5% of the data is used for testing
  shuffle=True, # Ensure that train and validation sets are the same across runs
  )
# Split into 80% training and 20% validation sets
train_dataset = test_dataset['train'].train_test_split(
  test_size=0.2, # 20% of the data is used for validation
  shuffle=True, # Ensure that train and test sets are the same across runs
  )
dataset = DatasetDict({
  'train': train_dataset['train'],
  'validation': train_dataset['test'],
  'test': test_dataset['test'],
  })
# 顯示處理後的資料
dataset

DatasetDict({
    train: Dataset({
        features: ['messages', 'user', 'assistant'],
        num_rows: 76
    })
    validation: Dataset({
        features: ['messages', 'user', 'assistant'],
        num_rows: 19
    })
    test: Dataset({
        features: ['messages', 'user', 'assistant'],
        num_rows: 5
    })
})

In [89]:
# 顯示前 first_n_data 筆資料
first_n_data = 5
pd.set_option('display.max_colwidth', None)
pd.DataFrame(dataset['test'].select(range(first_n_data)))

,messages,user,assistant
0,"[{'content': 'How do I create a RESTful API in Java?', 'role': 'user'}, {'content': 'Sorry, I cannot provide programming code or code snippets. I can assist you with the questions.', 'role': 'assistant'}]","{'content': 'How do I create a RESTful API in Java?', 'role': 'user'}","{'content': 'Sorry, I cannot provide programming code or code snippets. I can assist you with the questions.', 'role': 'assistant'}"
1,"[{'content': 'How do I implement a graph in C++?', 'role': 'user'}, {'content': 'I can't provide the code, but I can explain the structure.', 'role': 'assistant'}]","{'content': 'How do I implement a graph in C++?', 'role': 'user'}","{'content': 'I can't provide the code, but I can explain the structure.', 'role': 'assistant'}"
2,"[{'content': 'What is the difference between synchronous and asynchronous programming?', 'role': 'user'}, {'content': 'Synchronous programming executes tasks sequentially, while asynchronous programming allows tasks to run concurrently, improving performance and responsiveness.', 'role': 'assistant'}]","{'content': 'What is the difference between synchronous and asynchronous programming?', 'role': 'user'}","{'content': 'Synchronous programming executes tasks sequentially, while asynchronous programming allows tasks to run concurrently, improving performance and responsiveness.', 'role': 'assistant'}"
3,"[{'content': 'How do I create a RESTful API in Node.js?', 'role': 'user'}, {'content': 'Sorry, I cannot provide programming code or code snippets. I can assist you with the questions.', 'role': 'assistant'}]","{'content': 'How do I create a RESTful API in Node.js?', 'role': 'user'}","{'content': 'Sorry, I cannot provide programming code or code snippets. I can assist you with the questions.', 'role': 'assistant'}"
4,"[{'content': 'What is the difference between a hash table and a hash map?', 'role': 'user'}, {'content': 'A hash table is a data structure that maps keys to values using a hash function, while a hash map is a specific implementation of a hash table.', 'role': 'assistant'}]","{'content': 'What is the difference between a hash table and a hash map?', 'role': 'user'}","{'content': 'A hash table is a data structure that maps keys to values using a hash function, while a hash map is a specific implementation of a hash table.', 'role': 'assistant'}"


## 訓練參數

### 訓練設定

In [90]:
# 訓練相關設定
class Config(BaseModel):
  model_name: str = 'microsoft/Phi-3.5-mini-instruct'
  torch_dtype: Any = torch.bfloat16 # 半精度浮點數
  adam_epsilon: float = 1e-4 # 當使用半精度浮點數時，需要設定較大的 adam epsilon
  saved_model_path: str = os.path.join('saved_model', f'{__ipynb_file__}') # path to save the trained model
  saved_lora_path: str = os.path.join('saved_model', f'{__ipynb_file__}_lora') # path to save the trained LORA model
  batch_size: int = 2 # size of the input batch in training and evaluation
  gradient_accumulation_steps: int = 2 # number of updates steps to accumulate before performing a backward/update pass
  epochs: int = 25 # number of times to iterate over the entire training dataset
  lr: float = 2e-4 # learning rate, controls how fast or slow the model learns
  weight_decay: float = 0.01 # weight decay, helps the model stay simple and avoid overfitting by penalizing large weights.

  # 文本生成相關設定
  temperature: float = 0.1 # temperature for sampling
  max_new_tokens: int = 125 # 限制最大生成字數
  repetition_penalty: float = 1.5 # 重複機率, 1~2 之間, 1.0 (no penalty), 2.0 (maximum penalty)

  # LORA 相關設定
  rank: int = 128 # rank of the Lora layers
  lora_alpha: int = rank * 2 # alpha for Lora scaling.
  lora_dropout: float = 0.05 # dropout probability for Lora layers

if device.type == 'mps': # 方便在 Apple Silicon 上快速測試
  config = Config(
    torch_dtype=torch.float16, # 在 Apple Silicon 若使用預訓練模型 opt-125m 需要使用全精度浮點數，否則會出現錯誤
  )
else:
  config = Config()

## Fine-tuning 前的表現

### 載入預訓練分詞器 (Tokenizer)

In [91]:
# 透過預訓練模型取得 Tokenizer
tokenizer = AutoTokenizer.from_pretrained(
  config.model_name,
)
pprint(tokenizer)

LlamaTokenizerFast(name_or_path='microsoft/Phi-3.5-mini-instruct', vocab_size=32000, model_max_length=131072, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '<|endoftext|>', 'unk_token': '<unk>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=True, lstrip=False, single_word=False, normalized=False, special=False),
	32000: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32001: AddedToken("<|assistant|>", rstrip=True, lstrip=False, single_word=False, normalized=False, special=True),
	32002: AddedToken("<|placeholder1|>", rstrip=True, lstrip=False, single_word=False, normalized=False, special

* 如果沒有定義 `pad_token`，請定義一個 `pad_token`，並將其加入 Tokenizer 中。
* 如果 `padding_side` 不是 `right`，請將其設定為 `right`。

In [92]:
# Add pad_token to the tokenizer
if tokenizer.pad_token is None:
  tokenizer.pad_token = tokenizer.eos_token
  print('=== 設定 Padding Token ===')
  pprint(tokenizer)
# Make sure padding_side is 'right'
if tokenizer.padding_side != 'right':
  tokenizer.padding_side = 'right'
  print('=== 設定 Padding Side ===')
  pprint(tokenizer)

=== 設定 Padding Side ===
LlamaTokenizerFast(name_or_path='microsoft/Phi-3.5-mini-instruct', vocab_size=32000, model_max_length=131072, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '<|endoftext|>', 'unk_token': '<unk>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=True, lstrip=False, single_word=False, normalized=False, special=False),
	32000: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32001: AddedToken("<|assistant|>", rstrip=True, lstrip=False, single_word=False, normalized=False, special=True),
	32002: AddedToken("<|placeholder1|>", rstrip=True, lstrip=False, single_word=False, 

### 載入預訓練模型

由於 GPU 記憶體有限，我們將使用半精度進行模型 Fine-tuning。這邊需要留意，使用半精度進行 Fine-tuning 時，`TrainingArguments` 中的 `adam_epsilon` 需要設定為 `1e-4`。預設的 `adam_epsilon` 是 `1e-8`，這個值在半精度訓練時會出現問題。

透過 `AutoModelForCausalLM` 用於因果語言建模的自動類別，它可以載入不同的預訓練模型進行文本生成任務。

In [93]:
model = AutoModelForCausalLM.from_pretrained(
  config.model_name,
  torch_dtype=config.torch_dtype,
  # 這個參數用於優化內存使用，減少模型加載時的 CPU 內存佔用，特別是在內存有限的環境中非常有用。
  low_cpu_mem_usage=True,
).to(device)

Loading checkpoint shards: 100%|███████████████████████████████████████████████████████| 2/2 [00:13<00:00,  6.70s/it]


### 配置聊天樣本 (Chat Template)

在語言模型中添加特殊標記對於訓練聊天模型至關重要。這些標記被添加在對話中不同角色之間，例如 `user`、`assistant` 和 `system`，幫助模型識別對話的結構和流程。這種設置對於使模型在聊天環境中生成連貫且上下文適當的回應是必不可少的。

`trl` 中的 `setup_chat_format()` 函數可以輕鬆地為對話式 AI 任務設置模型和分詞器。

In [94]:
# Set up the chat format with default 'chatml' format
if tokenizer.chat_template is None:
  model, tokenizer = setup_chat_format(model, tokenizer)
  print('=== 設定 chat format ===')
  pprint(tokenizer)

由於我們使用的模型已經是一個聊天模型，我們不需要再次設置對話格式。

### 詠唱格式化 (Prompt Formatting)

定義詠唱 (Prompt) 格式，我們將創建一個格式化函數。

請注意，這次我們指定 `add_generation_prompt` 為 `True`，表示回應開始的標記。這確保了當模型生成文本時，它會寫出機器人的回應，而不是做一些意想不到的事情，比如繼續用戶的訊息。請記住，聊天模型仍然只是語言模型，它們被訓練來玩文字接龍，而聊天對它們來說只是一種特殊的文本！你需要用適當的控制標記來引導它們，讓它們知道應該做什麼。

In [95]:
def instruction_formatter(x, tokenize):
  if tokenize:
    return tokenizer.apply_chat_template(
      [x['user']],
      tokenize=tokenize,
      add_generation_prompt=True,
      return_tensors='pt',
      return_dict=True,
    ).to(device)
  else:
    return tokenizer.apply_chat_template(
      [x['user']],
      tokenize=tokenize,
      add_generation_prompt=True,
    )

`tokenize=False` 代表不進行 Tokenize，直接回傳原始文字，以及保留特殊標記。由於我們額外指定 `add_generation_prompt` 為 `True`，這將會在回應開始時加入特殊標記 `<|assistant|>`。

In [96]:
# tokenize=False 代表不進行 Tokenize，直接回傳原始文字
input = instruction_formatter(dataset['test'][0], tokenize=False)
pprint(input)

'<|user|>\nHow do I create a RESTful API in Java?<|end|>\n<|assistant|>\n'


In [97]:
# tokenize=True 代表進行 Tokenize，回傳 Tokenize 後的 ID 及 attention mask tensors
tokenized_input = instruction_formatter(dataset['test'][0], tokenize=True)
pprint(tokenized_input)

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='mps:0'),
 'input_ids': tensor([[32010,  1128,   437,   306,  1653,   263, 16759,  1319,  3450,   297,
          3355, 29973, 32007, 32001]], device='mps:0')}


當 `tokenize=True`，Tokenizer 回傳內容包含兩個主要部分：`input_ids` 和 `attention_mask`。以下是詳細解釋：

* `input_ids`: 是一個張量 (tensor)，包含了輸入文本的 token IDs。這些 IDs 是由 tokenizer 將文本轉換為數字表示後得到的。

* `attention_mask`: 同樣是一個張量，用於指示模型應該關注哪些位置。值為 1 的位置表示應該關注，值為 0 的位置表示應該忽略。在這個例子中，`attention_mask` 的值全為 1，表示模型應該關注所有位置。

In [98]:
# 透過 Tokenizer 的 decode 方法將 ID 轉換回文字，並列顯示出來
for id in tokenized_input['input_ids'][0]:
  print(f'{id} -> {tokenizer.decode([id])}')

32010 -> <|user|>
1128 -> How
437 -> do
306 -> I
1653 -> create
263 -> a
16759 -> REST
1319 -> ful
3450 -> API
297 -> in
3355 -> Java
29973 -> ?
32007 -> <|end|>
32001 -> <|assistant|>


### Fine-tuning 前的表現

#### 單筆演示生成回應

In [99]:
# 透過預訓練模型生成回應
output_ids = model.generate(
  **tokenized_input,
  temperature=config.temperature,
  max_new_tokens=config.max_new_tokens,
  repetition_penalty=config.repetition_penalty,
)

/Users/jonas/.pyenv/versions/learning-transformer-with-huggingface-3.11.11/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/Users/jonas/.pyenv/versions/learning-transformer-with-huggingface-3.11.11/lib/python3.11/site-packages/transformers/pytorch_utils.py:325: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_elements = torch.tensor(test_elements)


In [100]:
output_ids

tensor([[32010,  1128,   437,   306,  1653,   263, 16759,  1319,  3450,   297,
          3355, 29973, 32007, 32001, 26221,   385,  8543,   322, 11592, 11654,
          8159, 29749,   313,  1525,  1254, 29897,   773,   278,  7206, 13760,
          6890, 20789,  3196,  6576, 29889,  2266, 29915, 29879,   920,   366,
           508,   679,  4687, 29901,    13, 28956,  1645, 29871,  2115,   259,
          1053,  1638, 26355,   849, 16032,  5181,  9741,   363,   596,  2060,
         29892,  1316,   408,  6709, 29899,  4777,  9562,  2992,   636,  5215,
           419,  5575, 29936,   458,  3575,  5354,  4413,   470,   360,  4986,
         24802,  1244,   856,  3597,   770,  1619, 11713,  4873,   426,  1678,
           970,  2294,  1780,  1667, 29898,  1231,  2636,  6389,  2597,  4706,
          8427,  2677, 12893,   353,   716,   530, 16666,  3991, 10735,  3609,
          6004,  5126,  2141,   657,  2061,   890,   308,  9056, 13228,  1732,
         29903,   522, 29891, 29922,  1482,  5506, 1

In [101]:
# 將 output_ids 轉換為文字
output = tokenizer.decode(
  output_ids[0],
  skip_special_tokens=False, # 決定是否跳過特殊 token（例如，開始和結束標記）。
)

In [102]:
pprint(output)

('<|user|> How do I create a RESTful API in Java?<|end|><|assistant|> Creating '
 'an efficient and secure Restfull Api (REST) using the Spring Boot framework '
 "involves several steps. Here's how you can get started:\n"
 '```java  java   import org.*; // Import necessary packages for your project, '
 'such as spring-boot libraries etc..import com.*;// Your domain classes or '
 'DTO imports here...public class MyApiApplication {    public static void '
 'main(String[] args){        ApplicationContext ctx = new '
 'AnnotationConfigServletWebServerFactory().getObject();         HttpSecurity '
 'httpSecty=newHttpBasic() .and(){            antMatchers("/api/**").')


只取得生成的文字, 即 `<|assistant|>` 之後的文字

In [103]:
# 只取得生成的文字, 即 <|assistant|> 之後的文字
pprint(output.split('<|assistant|>')[1].strip())

('Creating an efficient and secure Restfull Api (REST) using the Spring Boot '
 "framework involves several steps. Here's how you can get started:\n"
 '```java  java   import org.*; // Import necessary packages for your project, '
 'such as spring-boot libraries etc..import com.*;// Your domain classes or '
 'DTO imports here...public class MyApiApplication {    public static void '
 'main(String[] args){        ApplicationContext ctx = new '
 'AnnotationConfigServletWebServerFactory().getObject();         HttpSecurity '
 'httpSecty=newHttpBasic() .and(){            antMatchers("/api/**").')


#### 批次處理模型表現

初步了解如何生成模型的回應，我們將定義一個 `generate()` 函數來生成模型的回應。這個函數接受一個輸入文本，並生成模型的回應。藉由這個函數，我們可以批次處理資料。


In [104]:
# 將以上程式碼整理成一個函式，方便我們批次處理資料
def generator(x, model):
  tokenized_input = instruction_formatter(x, tokenize=True)
  output_ids = model.generate(
    **tokenized_input,
    temperature=config.temperature,
    max_new_tokens=config.max_new_tokens,
    repetition_penalty=config.repetition_penalty,
  )
  output = tokenizer.decode(output_ids[0], skip_special_tokens=False)
  return output.split('<|assistant|>')[1].strip()

In [105]:
# 這個步驟可能會花費一些時間，所以我們只處理前 first_n_data 筆資料
first_n_dataset = dataset['test'].select(range(first_n_data))

# 移除 messages 欄位
first_n_dataset = first_n_dataset.remove_columns('messages')

# 透過預訓練模型生成回應，將其新增到 pt_response 欄位中
pt_response = []
for x in first_n_dataset:
  pt_response.append(generator(x, model))

first_n_df = pd.DataFrame(first_n_dataset)
first_n_df['pt_response'] = pt_response

In [106]:
# 顯示預訓練模型預測結果
pd.set_option('display.max_colwidth', None)
first_n_df

,user,assistant,pt_response
0,"{'content': 'How do I create a RESTful API in Java?', 'role': 'user'}","{'content': 'Sorry, I cannot provide programming code or code snippets. I can assist you with the questions.', 'role': 'assistant'}","Creating an efficient and secure Restfull Api (REST) using the Spring Boot framework involves several steps. Here's how you can get started:\n```java java import org.*; // Import necessary packages for your project, such as spring-boot libraries etc..import com.*;// Your domain classes or DTO imports here...public class MyApiApplication { public static void main(String[] args){ ApplicationContext ctx = new AnnotationConfigServletWebServerFactory().getObject(); HttpSecurity httpSecty=newHttpBasic() .and(){ antMatchers(""/api/**"")."
1,"{'content': 'How do I implement a graph in C++?', 'role': 'user'}","{'content': 'I can't provide the code, but I can explain the structure.', 'role': 'assistant'}","Implementing an entire Graph data structure from scratch can be quite involved, but here's how you could start with the basics of creating and managing nodes (vertices) within your own simple undirected or directed graphs using adjacency lists.\n Here is some basic code to get started: 📝 **Graph Class Definition** ```cpp #include <iostream> // For input/output operations class Vertex { public : int id; std::vector<Vertex* > neighbors ; }; /* A vertex object contains its unique identifier"
2,"{'content': 'What is the difference between synchronous and asynchronous programming?', 'role': 'user'}","{'content': 'Synchronous programming executes tasks sequentially, while asynchronous programming allows tasks to run concurrently, improving performance and responsiveness.', 'role': 'assistant'}","Synchronous (sometimes referred to as ""blocking"") vs. Asynchrony are two fundamental approaches in computer science for handling tasks, especially when dealing with operations that may take an indeterminate amount of time such like I/O-bound or network requests which can significantly impact performance if not managed correctly:\n 1) **Synchronous Programming** involves executing one operation at a given point before moving on sequentially through subsequent instructions until another task needs attention—think traditional procedural code where each step waits its predecessor'll complete execution without interruption unless explicitly programmed otherwise using mult"
3,"{'content': 'How do I create a RESTful API in Node.js?', 'role': 'user'}","{'content': 'Sorry, I cannot provide programming code or code snippets. I can assist you with the questions.', 'role': 'assistant'}","Creating an efficient and scalable Restfull Api with Express (NodeJS) involves several steps:\n1- Setting up your project environment by installing necessary packages such as `express`, `@types/node` for type checking, body parser middleware (`bodyParser`), cors package to handle cross origin requests(`cors`) etc., using npm or yarn commands like so; 📝 ```bash $ sudo apt install node ➡️ Install NPM if not installed already `$ curl -sL https://debianshadowcdnopseshadowdotcom//install_npm | bash"
4,"{'content': 'What is the difference between a hash table and a hash map?', 'role': 'user'}","{'content': 'A hash table is a data structure that maps keys to values using a hash function, while a hash map is a specific implementation of a hash table.', 'role': 'assistant'}","The terms ""hash-table"" (or simply, “hashtable”) often refer to essentially similar concepts in computer science but can be used slightly differently depending on context or programming language. Here's an explanation that clarifies their relationship:\n 1 . **Hash Table** - This term refers broadly across different implementations of data structures designed for efficient lookup operations based upon key values mapped through hashing functions into buckets where these keys are stored along with associated value(ies). It encompasses various underlying mechanisms like linked lists

## 訓練模型

### LoRA 的訓練策略

LoRA（Low-Rank Adaptation）是一種用於訓練大型語言模型的技術，旨在提高訓練效率並減少計算資源的需求。以下是為何需要透過LoRA訓練的一些原因：

* 降低計算成本：LoRA 通過將模型的權重矩陣分解為低秩矩陣，顯著減少了參數的數量，從而降低了計算成本和內存需求。

* 加速訓練速度：由於參數數量減少，LoRA 可以加速模型的訓練過程，使得在相同的硬件資源下能夠更快地完成訓練。

![](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/peft/lora_diagram.png)

我們先來觀察預訓練模型可訓練的參數量，其數量相當龐大，所以需要透過 Low Rank Adaptation (LoRA) 來降低參數量。

In [107]:
# 查看預訓練模型可訓練的參數量，其數量相當龐大，所以需要透過 Low Rank Adaptation (LoRA) 來降低參數量
print('Parameters: {:,}, Trainable Parameters: {:,}'.format(
  model.num_parameters(),
  model.num_parameters(only_trainable=True)))

Parameters: 3,821,079,552, Trainable Parameters: 3,821,079,552


#### LoRA 配置

* `task_type`: TaskType.CAUSAL_LM 指定任務類型為因果語言模型 (Causal Language Model)。

* `rank`: 是低秩矩陣的秩(rank)，它決定了 LoRA 層的參數數量。較低的 `r` 值意味著較少的參數，從而減少了模型的計算和存儲需求。具體來說，LoRA 通過將全連接層的權重矩陣分解為兩個低秩矩陣來實現參數高效化。`r` 值越小，這兩個低秩矩陣的維度越小，這個練習我們採用 128。

* `lora_alpha`: 是一個縮放因子，用於調整 LoRA 層的輸出。它控制了低秩矩陣的影響力。較高的 `lora_alpha` 值會增加 LoRA 層的影響力，也就是說值越高，越容易把大模型既有的能力給覆蓋掉。具體來說，LoRA 層的輸出會乘以這個縮放因子，這個練習我們採用常見的比例為 `rank` 的兩倍。

* `lora_dropout`: 是一個丟棄率，用於在訓練過程中隨機丟棄 LoRA 層的一部分輸出。這有助於防止過擬合，並提高模型的泛化能力。例如，`lora_dropout` 設置為 0.1 表示在每次前向傳播中，有 10% 的 LoRA 層輸出會被隨機設置為零。

* `target_module`: 指定了應用 LoRA 的目標模塊。這通常是模型中的某些特定層或子模塊，例如 Transformer 模型中的注意力層，可以透過 `model.named_parameters` 查看。通過指定 `target_module`，你可以靈活地選擇在哪些層應用 LoRA，以便在保持模型性能的同時減少參數數量。

> 廣為周知的模型當未指定 `target_module`，透過 `get_peft_model` 加載 Lora 適配模型時，會自動設定。
> 可以先嘗試不指定，若出現錯誤再試著設定注意力相關的參數層。


In [108]:
# LoRA 配置
lora_config = LoraConfig(
  task_type=TaskType.CAUSAL_LM,
  r=config.rank,
  lora_alpha=config.lora_alpha,
  lora_dropout=config.lora_dropout,
  # Phi3ForCausalLM need to specify the target_modules beforehand
  target_modules=['qkv_proj'],
)

pprint(lora_config)

LoraConfig(task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>,
           peft_type=<PeftType.LORA: 'LORA'>,
           auto_mapping=None,
           base_model_name_or_path=None,
           revision=None,
           inference_mode=False,
           r=128,
           target_modules={'qkv_proj'},
           exclude_modules=None,
           lora_alpha=256,
           lora_dropout=0.05,
           fan_in_fan_out=False,
           bias='none',
           use_rslora=False,
           modules_to_save=None,
           init_lora_weights=True,
           layers_to_transform=None,
           layers_pattern=None,
           rank_pattern={},
           alpha_pattern={},
           megatron_config=None,
           megatron_core='megatron.core',
           loftq_config={},
           eva_config=None,
           use_dora=False,
           layer_replication=None,
           runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False),
           lora_bias=False)


#### 加載 LoRA 適配模型

搭配預訓模型及 LoRA 配置，我們可以加載 LoRA 適配模型。我們可以觀察受到降維影響的模型層。

In [109]:
# 加載 LoRA 適配模型
peft_model = get_peft_model(
  model, # 預訓練模型
  lora_config, # LoRA 配置
)

In [110]:
pprint(lora_config)

LoraConfig(task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>,
           peft_type=<PeftType.LORA: 'LORA'>,
           auto_mapping=None,
           base_model_name_or_path='microsoft/Phi-3.5-mini-instruct',
           revision=None,
           inference_mode=False,
           r=128,
           target_modules={'qkv_proj'},
           exclude_modules=None,
           lora_alpha=256,
           lora_dropout=0.05,
           fan_in_fan_out=False,
           bias='none',
           use_rslora=False,
           modules_to_save=None,
           init_lora_weights=True,
           layers_to_transform=None,
           layers_pattern=None,
           rank_pattern={},
           alpha_pattern={},
           megatron_config=None,
           megatron_core='megatron.core',
           loftq_config={},
           eva_config=None,
           use_dora=False,
           layer_replication=None,
           runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False),
           lora_bias=False)


#### LoRA 適配模型

加載 LoRA 適配模型後, 觀察受 LoRA 影響的模型參數

In [111]:
peft_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Phi3ForCausalLM(
      (model): Phi3Model(
        (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
        (embed_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-31): 32 x Phi3DecoderLayer(
            (self_attn): Phi3SdpaAttention(
              (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
              (qkv_proj): lora.Linear(
                (base_layer): Linear(in_features=3072, out_features=9216, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=128, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=128, out_features=9216, bias=False)
                )
                (lora_embedding_A): ParameterDict()


由於我們指定的 `target_module` 是 `qkv_proj`, 因此所有注意力層受到 LoRA 的影響。

```json
  (qkv_proj): lora.Linear(
    (base_layer): Linear(in_features=3072, out_features=9216, bias=False)
    (lora_dropout): ModuleDict(
      (default): Dropout(p=0.05, inplace=False)
    )
    (lora_A): ModuleDict(
      (default): Linear(in_features=3072, out_features=128, bias=False)
    )
    (lora_B): ModuleDict(
      (default): Linear(in_features=128, out_features=9216, bias=False)
    )
    (lora_embedding_A): ParameterDict()
    (lora_embedding_B): ParameterDict()
    (lora_magnitude_vector): ModuleDict()
  )
```              

#### 調整 LoRA 精度

LoRA 適配模型的精度是 `torch.float32`，我們可以透過 `model.half()` 將其轉換為半精度。

In [112]:
if config.torch_dtype == torch.float16 or config.torch_dtype == torch.bfloat16:
  peft_model = peft_model.half() # 轉換為半精度浮點數

In [113]:
# 獲取 LoRA 模型參數名稱及型態，確認是否使用半精度浮點數
for name, param in peft_model.named_parameters():
  print(f'{name}: {param.dtype}')

base_model.model.model.embed_tokens.weight: torch.float16
base_model.model.model.layers.0.self_attn.o_proj.weight: torch.float16
base_model.model.model.layers.0.self_attn.qkv_proj.base_layer.weight: torch.float16
base_model.model.model.layers.0.self_attn.qkv_proj.lora_A.default.weight: torch.float16
base_model.model.model.layers.0.self_attn.qkv_proj.lora_B.default.weight: torch.float16
base_model.model.model.layers.0.mlp.gate_up_proj.weight: torch.float16
base_model.model.model.layers.0.mlp.down_proj.weight: torch.float16
base_model.model.model.layers.0.input_layernorm.weight: torch.float16
base_model.model.model.layers.0.post_attention_layernorm.weight: torch.float16
base_model.model.model.layers.1.self_attn.o_proj.weight: torch.float16
base_model.model.model.layers.1.self_attn.qkv_proj.base_layer.weight: torch.float16
base_model.model.model.layers.1.self_attn.qkv_proj.lora_A.default.weight: torch.float16
base_model.model.model.layers.1.self_attn.qkv_proj.lora_B.default.weight: torch.

經過 `model.half()` 轉換後，LoRA 適配模型的權重也變成半精度。

```shell
base_model.model.model.layers.0.self_attn.qkv_proj.base_layer.weight: torch.float16
base_model.model.model.layers.0.self_attn.qkv_proj.lora_A.default.weight: torch.float16
base_model.model.model.layers.0.self_attn.qkv_proj.lora_B.default.weight: torch.float16
```

訓練參數量也從原先 3B 大大減少為 50M。

In [114]:
# 查看可訓練的參數量
peft_model.print_trainable_parameters()

trainable params: 50,331,648 || all params: 3,871,411,200 || trainable%: 1.3001


### 詠唱格式化 (Prompt Formatting)

有別於先前的詠唱格式，這次我們將包含 `assistant` 的回應，以便作為標注資料供模型訓練。由於已經包含 `assistant`，這次我們指定 `add_generation_prompt` 為 `False`，省卻回應開始的標記。

另一個差異是，這個函式預設不會進行 tokenize，會直接回傳原始文字。

In [115]:
def instruction_completion_formatter(x, tokenize: bool = False):
  return tokenizer.apply_chat_template(
    x['messages'],
    tokenize=tokenize,
    add_generation_prompt=False,
  )

In [116]:
pprint(instruction_completion_formatter(dataset['train'][0]))

('<|user|>\n'
 'What is the difference between a hash set and a hash map?<|end|>\n'
 '<|assistant|>\n'
 'A hash set is a collection of unique elements, while a hash map is a '
 'collection of key-value pairs.<|end|>\n'
 '<|endoftext|>')


### 資料校對器 (Data Collator)

In [117]:
# 定義回應開始的標記
response_template = '<|assistant|>'

# 設定 DataCollatorForCompletionOnlyLM
data_collator = DataCollatorForCompletionOnlyLM(
  tokenizer=tokenizer,
  response_template=response_template,
)

In [118]:
# 展示 DataCollatorForCompletionOnlyLM 的輸出, 標籤以 -100 表示在損失函數中不會被考慮
batch = data_collator([instruction_completion_formatter(dataset['train'][i], True) for i in range(first_n_data)])
pprint(batch)

{'input_ids': tensor([[32010,  1724,   338,   278,  4328,  1546,   263,  6608,   731,   322,
           263,  6608,  2910, 29973, 32007, 32001,   319,  6608,   731,   338,
           263,  4333,   310,  5412,  3161, 29892,  1550,   263,  6608,  2910,
           338,   263,  4333,   310,  1820, 29899,  1767, 11000, 29889, 32007,
         32000, 32000, 32000, 32000],
        [32010,  1815,   366,  2436,   263,   740,   304,  2656,   263,  1051,
           297,  5132, 29973, 32007, 32001,  8221, 29892,   306,  2609,  3867,
          8720,   775,   470,   775,  9830, 27421, 29889,   306,   508,  6985,
           366,   411,   278,  5155, 29889, 32007, 32000, 32000, 32000, 32000,
         32000, 32000, 32000, 32000],
        [32010,  1815,   366,  2436,   263,   740,   304,  1284,   278, 14176,
          3619,  8572,   272,   313, 29954,  6530, 29897,   297,  8286, 29973,
         32007, 32001,  8221, 29892,   306,  2609,  3867,  8720,   775,   470,
           775,  9830, 27421, 29889,   30

In [119]:
# 透過 Tokenizer 的 decode 方法將 ID 轉換回文字，並列標籤顯示出來
for idx in range(first_n_data):
  input_ids = batch['input_ids'][idx]
  labels_ids = batch['labels'][idx]
  input = [tokenizer.decode(id) for id in input_ids]
  labels = ['-'] * len(input_ids)
  for i, id in enumerate(labels_ids):
    if id != -100:
      labels[i] = tokenizer.decode(id)
  pprint(f'input: {input}')
  pprint(f'label: {labels}')

("input: ['<|user|>', 'What', 'is', 'the', 'difference', 'between', 'a', "
 "'hash', 'set', 'and', 'a', 'hash', 'map', '?', '<|end|>', '<|assistant|>', "
 "'A', 'hash', 'set', 'is', 'a', 'collection', 'of', 'unique', 'elements', "
 "',', 'while', 'a', 'hash', 'map', 'is', 'a', 'collection', 'of', 'key', '-', "
 "'value', 'pairs', '.', '<|end|>', '<|endoftext|>', '<|endoftext|>', "
 "'<|endoftext|>', '<|endoftext|>']")
("label: ['-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-', "
 "'-', '-', '-', 'A', 'hash', 'set', 'is', 'a', 'collection', 'of', 'unique', "
 "'elements', ',', 'while', 'a', 'hash', 'map', 'is', 'a', 'collection', 'of', "
 "'key', '-', 'value', 'pairs', '.', '<|end|>', '-', '-', '-', '-']")
("input: ['<|user|>', 'Can', 'you', 'write', 'a', 'function', 'to', 'sort', "
 "'a', 'list', 'in', 'Python', '?', '<|end|>', '<|assistant|>', 'Sorry', ',', "
 "'I', 'cannot', 'provide', 'programming', 'code', 'or', 'code', 'sni', "
 "'ppets', '.', 'I', 'can', 'assist', 

你可以清楚觀察到，損失函數不會去關注包含 `<|assistant|>` 之前的部分，這樣可以讓模型專注於生成 `<|assistant|>` 之後的回應。

考量批次訓練不同長度的序列需要填充到相同的長度，以便能夠在同一批次中進行處理。Data Collator 自動進行適當的填充，填充的部分亦不會參與損失計算。

### 訓練參數設定

用於設定訓練過程中的各種參數，如學習率、批次大小、梯度累積步數、訓練 epoch 數、權重衰減等。

* `output_dir` 指定了訓練輸出的目錄。
* `logging_steps` 訓練時的日誌步數，決定每隔多少步輸出一次訓練日誌。
* `eval_strategy` 和 `save_strategy` 設定為 'steps'，表示每 `logging_steps` 個 steps 都會進行評估和儲存。
* `load_best_model_at_end` 設定為 `True`，表示訓練結束後會載入最佳模型。
* `report_to` 禁用 wandb 報告，適用於 Colab 環境，避免需要配置 wandb。
* `adam_epsilon` Adam 優化器的 epsilon 值，當使用半精度浮點數時需要設定較大的值以穩定訓練。
* `packing` 當使用 DataCollatorForCompletionOnlyLM 時禁用 packing，這是特定於數據整理器的設定。
* `save_total_limit` 最多儲存 5 個 checkpoints，控制儲存的模型檔案數量以節省磁碟空間。

In [120]:
training_args = SFTConfig(
  output_dir='sample_data/train_output_qa', # 訓練輸出目錄
  learning_rate=config.lr, # 學習率
  per_device_train_batch_size=config.batch_size, # 每個設備的訓練批次大小
  per_device_eval_batch_size=config.batch_size, # 每個設備的評估批次大小
  gradient_accumulation_steps=config.gradient_accumulation_steps, # 梯度累積步數
  logging_steps=50, # 訓練時的日誌步數, 預設每 500 步輸出一次日誌
  num_train_epochs=config.epochs, # 訓練的總 epoch 數
  weight_decay=config.weight_decay, # 權重衰減
  eval_strategy='steps', # 評估策略
  save_strategy='steps', # 儲存策略
  load_best_model_at_end=True,
  report_to='none', # 禁用 wandb 報告 (Colab 環境預設需要 wandb)
  adam_epsilon=config.adam_epsilon, # 當使用半精度浮點數時，需要設定較大的 adam epsilon
  packing=False, # 當使用 DataCollatorForCompletionOnlyLM 時禁用 packing
  save_total_limit=5, # 最多儲存 5 個 checkpoints
)

### 訓練器初始化

用於初始化訓練器，並開始訓練模型。

* `model` 是要訓練的模型。
* `tokenizer` 是用於處理文本的分詞器。
* `train_dataset` 是訓練數據集。
* `formatting_func` 是用於格式化數據的函數。
* `data_collator` 是用於整理數據的數據整理器。

In [121]:
trainer = SFTTrainer(
    model=peft_model, # 要訓練的模型
    tokenizer=tokenizer, # 使用的分詞器
    args=training_args, # 訓練參數
    train_dataset=dataset['train'], # 訓練數據集
    eval_dataset=dataset['validation'], # 驗證數據集
    formatting_func=instruction_completion_formatter, # 格式化函數
    data_collator=data_collator, # 數據整理器
)

/var/folders/j8/hkxkfjqd58j9t_718vfr4tjw0000gn/T/ipykernel_6831/1766244151.py:1: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(
Map: 100%|██████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 4697.42 examples/s]


### 開始訓練

In [122]:
# 開始訓練，這可能需要一些時間
trainer.train()

Step,Training Loss,Validation Loss
50,0.422000,0.663794
100,0.107200,0.590753
150,0.034300,0.789255
200,0.015200,0.888242
250,0.002000,0.992421
300,0.000200,1.012923
350,0.000200,1.019349
400,0.000100,1.021163
450,0.000200,1.021307


TrainOutput(global_step=475, training_loss=0.06120919820019289, metrics={'train_runtime': 758.746, 'train_samples_per_second': 2.504, 'train_steps_per_second': 0.626, 'total_flos': 1937268855853056.0, 'train_loss': 0.06120919820019289, 'epoch': 25.0})

#### 保存 LoRA 模型參數

In [123]:
# 保存 Lora 参数
peft_model.save_pretrained(
  config.saved_lora_path,
  # warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
  save_embedding_layers=True,
)

#### 保存 Tokenizer

In [124]:
# 保存 Tokenizer
tokenizer.save_pretrained(config.saved_model_path)

('saved_model/fine-tune-a8013a93-85f5-4b57-9a85-385b291713e2/tokenizer_config.json',
 'saved_model/fine-tune-a8013a93-85f5-4b57-9a85-385b291713e2/special_tokens_map.json',
 'saved_model/fine-tune-a8013a93-85f5-4b57-9a85-385b291713e2/tokenizer.model',
 'saved_model/fine-tune-a8013a93-85f5-4b57-9a85-385b291713e2/added_tokens.json',
 'saved_model/fine-tune-a8013a93-85f5-4b57-9a85-385b291713e2/tokenizer.json')

### 釋放資源

In [125]:
# import garbage collector
import gc

# 釋放 GPU 記憶體
del trainer
del tokenizer

peft_model.to('cpu')
del peft_model

torch.cuda.empty_cache()

gc.collect()

284

## 評估微調模型

### 載入微調分詞器 (Tokenizer)

從已經完成訓練的模型取得 Tokenizer，可以留意這個訓練時保存下來的 Tokenizer 仍保有訓練時的設定，包涵 `pad_token` 和 `padding_side`。

In [126]:
tokenizer = AutoTokenizer.from_pretrained(
  config.saved_model_path
)

### 載入微調後模型

In [127]:
ft_model = PeftModel.from_pretrained(
  model, # 預訓練模型
  config.saved_lora_path, # LoRA 適配模型
  # 這個參數用於優化內存使用，減少模型加載時的 CPU 內存佔用，特別是在內存有限的環境中非常有用。
  low_cpu_mem_usage=True,
  torch_dtype=config.torch_dtype,
).to(device)

### Fine-tuning 後的表現

In [128]:
# 透過微調模型生成回應，將其新增到 ft_response 欄位中
ft_response = []
for x in first_n_dataset:
  ft_response.append(generator(x, ft_model))

first_n_df['ft_response'] = ft_response

/Users/jonas/.pyenv/versions/learning-transformer-with-huggingface-3.11.11/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/Users/jonas/.pyenv/versions/learning-transformer-with-huggingface-3.11.11/lib/python3.11/site-packages/transformers/pytorch_utils.py:325: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_elements = torch.tensor(test_elements)


In [129]:
# 顯示微調模型預測結果
pd.set_option('display.max_colwidth', None)
first_n_df

,user,assistant,pt_response,ft_response
0,"{'content': 'How do I create a RESTful API in Java?', 'role': 'user'}","{'content': 'Sorry, I cannot provide programming code or code snippets. I can assist you with the questions.', 'role': 'assistant'}","Creating an efficient and secure Restfull Api (REST) using the Spring Boot framework involves several steps. Here's how you can get started:\n```java java import org.*; // Import necessary packages for your project, such as spring-boot libraries etc..import com.*;// Your domain classes or DTO imports here...public class MyApiApplication { public static void main(String[] args){ ApplicationContext ctx = new AnnotationConfigServletWebServerFactory().getObject(); HttpSecurity httpSecty=newHttpBasic() .and(){ antMatchers(""/api/**"").","Sorry, I cannot provide programming code or codes. However, I can guide you on the principles and methodologies involved. Would you like to know more about it?<|end|>"
1,"{'content': 'How do I implement a graph in C++?', 'role': 'user'}","{'content': 'I can't provide the code, but I can explain the structure.', 'role': 'assistant'}","Implementing an entire Graph data structure from scratch can be quite involved, but here's how you could start with the basics of creating and managing nodes (vertices) within your own simple undirected or directed graphs using adjacency lists.\n Here is some basic code to get started: 📝 **Graph Class Definition** ```cpp #include <iostream> // For input/output operations class Vertex { public : int id; std::vector<Vertex* > neighbors ; }; /* A vertex object contains its unique identifier","Sorry, I cannot provide programming code or code snippets. I can assist you with the questions.<|end|>"
2,"{'content': 'What is the difference between synchronous and asynchronous programming?', 'role': 'user'}","{'content': 'Synchronous programming executes tasks sequentially, while asynchronous programming allows tasks to run concurrently, improving performance and responsiveness.', 'role': 'assistant'}","Synchronous (sometimes referred to as ""blocking"") vs. Asynchrony are two fundamental approaches in computer science for handling tasks, especially when dealing with operations that may take an indeterminate amount of time such like I/O-bound or network requests which can significantly impact performance if not managed correctly:\n 1) **Synchronous Programming** involves executing one operation at a given point before moving on sequentially through subsequent instructions until another task needs attention—think traditional procedural code where each step waits its predecessor'll complete execution without interruption unless explicitly programmed otherwise using mult","Synchronous programming involves operations that run in sequence, while asynchronous programming allows for tasks to be executed concurrently.<|end|>"
3,"{'content': 'How do I create a RESTful API in Node.js?', 'role': 'user'}","{'content': 'Sorry, I cannot provide programming code or code snippets. I can assist you with the questions.', 'role': 'assistant'}","Creating an efficient and scalable Restfull Api with Express (NodeJS) involves several steps:\n1- Setting up your project environment by installing necessary packages such as `express`, `@types/node` for type checking, body parser middleware (`bodyParser`), cors package to handle cross origin requests(`cors`) etc., using npm or yarn commands like so; 📝 ```bash $ sudo apt install node ➡️ Install NPM if not installed already `$ curl -sL https://debianshadowcdnopseshadowdotcom//install_npm | bash","Sorry, I cannot provide programming code or code snippets. I can assist you with the questions.<|end|>"
4,"{'content': 'What is the difference between a hash table and a hash map?', 'role': 'user'}","{'content': 'A hash table is a data structure that maps keys to values using a hash function, while a hash map is a specific implementation of a hash table.', 'role': 'assistant'}","The terms ""hash-table"" (or simply, “hashta